In [1]:
import daw_readout
import process_data
import pandas as pd
import numpy as np
import h5py
from tqdm import tqdm
import sys
import time
import os

In [2]:
#file_list = 'runlist/darkrate_monitor_with_GECO2020'
#file_list = 'runlist/LED_1.7V_11ns_400_ratio_runs'
file_list = 'runlist/LED_1.7V_7.5ns_0.5_ratio_runs'

In [3]:
path_save =""
winfo =[]
#i=0
#tmp_ch = 0
with open(file_list, 'r') as list:
    for line in list:    
        rawfilename = line.rstrip('\n')[17 :] 
        rawdata = daw_readout.DAWDemoWaveParser(rawfilename)     
        for wave in tqdm(rawdata) :            
            ch = wave.Channel
            ttt = wave.Timestamp
            base = wave.Baseline
            pulse = wave.Waveform
            st, minp, ed = process_data.pusle_index(pulse)
            ht = base - wave.Waveform[minp]            
            #rst = minp  - st
            #flt = ed - minp
            #maxp = base - np.max(pulse)
            #asy = ht / (ht + maxp)
            area = process_data.pusle_area(pulse, st, ed, base)
            area_fixlen = process_data.pulse_area_fix_len(pulse, st, 10, base)
            winfo.append({         
                #'EvtID': i,
                'Ch':ch,
                'TTT':ttt,   ## Trigger time tag
                'Baseline': base, 
                'Hight': ht, 
                #'Width': ed -st,  ## pusle width
                #'Minp': minp,     ## min point
                #'Rst' : rst,      ## rise time of pulse    
                #'Flt' : flt,      ## full time of pulse  
                #'Asym' : asy,     ## asymmetry of pulse
                'Area': area,     ## area of pulse cal by dynamic range
                'AreaFixlen': area_fixlen,    ## area of pulse cal by fix length of 10
                #'Wave': pulse,
            })            
            #if(ch != tmp_ch):
            #    i += 1
            #tmp_ch = ch
        file_tag = line.rstrip('\n')[17 :].rstrip('.bin')[24:][: -13]  
        path_save = "outnpy/{}.h5".format(file_tag)

292006it [00:33, 8700.06it/s] 
3348912it [01:35, 35061.87it/s]
3348816it [01:35, 35066.43it/s]
113100it [00:03, 35212.65it/s]


In [6]:
path_save

'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240820_LED_1.7V_7.5ns_run.h5'

In [7]:
df = pd.DataFrame(winfo)
process_data.write_to_hdf5(df, path_save)

OverflowError: value too large to convert to int